In [27]:
import pickle
import sys
import os
import numpy as np

# Get the path of the current script (or current working directory)
parent_dir = os.path.dirname(os.getcwd())
# Append the parent directory to sys.path
sys.path.append(parent_dir)

import src
from src.IndependentSteps import Pycromanager2NativeDataType, Make_Output_Dir_JF, Make_Analysis_Dir_JF, \
                                    ConsolidateImageShapes, TrimZSlices, AutomaticSpotDetection_JF

from src.SequentialSteps import CellSegmentationStepClass_JF, BIGFISH_SpotDetection, SimpleCellposeSegmentaion

from src.FinalizationSteps import BuildPDFReport, SaveSpotDetectionResults, SaveMasksToAnalysis, SendAnalysisToNAS, \
                                    DeleteTempFiles, TrackPyAnlaysis

                                    

from src import Experiment, Settings, ScopeClass, DataContainer, Pipeline

def save_dict_to_file(dic, location):
    f = open(location,'w')
    f.write(str(dic))
    f.close()

In [28]:
pipeline_package_name = 'IntronDiffusion_TrackpyDiffusion'

# Initialize Dataclasses

In [29]:
settings = Settings(return_data_to_NAS = 1,
    NUMBER_OF_CORES = 4,
    save_files = 1,
    user_select_number_of_images_to_run = 100_000,
    download_data_from_NAS = 0 ,  # 0 for local, 1 for NAS
    connection_config_location = r"C:\Users\Jack\Desktop\config.yml", # r"/home/formanj/FISH_Processing_JF/FISH_Processing/config.yml"
    share_name = 'share',
    display_plots = False)

settings.bigfish_alpha = 0.5
settings.CLUSTER_RADIUS = 500

In [30]:
experiment = Experiment(initial_data_location=None,
                        nucChannel=[0],
                        cytoChannel=None,
                        FISHChannel=None,
                        voxel_size_z=300)

In [31]:
scope = ScopeClass( voxel_size_yx=130,
                    spot_z=500,
                    spot_yx=360,
                    microscope_saving_format='pycromanager'
                )

In [32]:
data = DataContainer()

In [33]:
setting_dict = settings.__dict__
experiment_dict = experiment.__dict__
scope_dict = scope.__dict__


# Initialize Steps

In [34]:
PrePipelineSteps = [Pycromanager2NativeDataType(), 
                   Make_Output_Dir_JF(), 
                   Make_Analysis_Dir_JF(), 
                   ConsolidateImageShapes(), 
                   AutomaticSpotDetection_JF()]

In [35]:
PipelineSteps = [SimpleCellposeSegmentaion(), 
                 BIGFISH_SpotDetection()]

In [36]:
PostPipelineSteps = [SaveSpotDetectionResults(), 
                     TrackPyAnlaysis(), 
                     BuildPDFReport(), 
                     SaveMasksToAnalysis(), 
                     SendAnalysisToNAS(), 
                     DeleteTempFiles()]

# Combine into single Dict

In [37]:
pipeline_dict = {'settings': setting_dict, 
                 'experiment': experiment_dict, 
                 'scope': scope_dict, 
                 'data': data.__dict__,
                 'independentSteps': [i.__class__.__name__ for i in PrePipelineSteps],
                 'sequentialSteps': [i.__class__.__name__ for i in PipelineSteps],
                 'finalizationSteps': [i.__class__.__name__ for i in PostPipelineSteps]}

In [38]:
# Save the instances to pickle files
package_dir = os.path.join(os.getcwd(), 'Pipelines')
os.makedirs(package_dir, exist_ok=True)

save_dict_to_file(pipeline_dict, os.path.join(package_dir, f'{pipeline_package_name}.txt'))



In [39]:
os.path.join(package_dir, f'{pipeline_package_name}.txt')

'c:\\Users\\ericron.COLOSTATE\\GitHub\\FISH_Processing\\Pipelines\\IntronDiffusion_TrackpyDiffusion.txt'